In [1]:
from IPython.display import Audio

import torch
import torch.nn as nn
import torchaudio
from utils import *
from feature_encoder import ConvFeatureExtractionModel
import math

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
x = []
for i in range(1,11):
    path = f"data/mp3_train_files/Gould/Gould - WTC_clip_{i}.mp3"
    waveform,sample_rate = torchaudio.load(path)
    waveform = torch.mean(waveform, dim=0).unsqueeze(0)
    waveform = normalize_tensor(waveform)
    x.append(waveform)


x = torch.cat(x)

x.shape

torch.Size([10, 220500])

In [4]:
#plot_waveform(x, sample_rate)

In [44]:
dim = 512 # quadratic to number of params in conv... double dim quadruple params

conv_feature_layers = (
    [(dim, 10, 2)] * 3 + [(dim, 3, 2)] * 5 + [(dim, 2, 2)] + [(dim, 2, 2)]
)  # (dim,kernel width, strides) # kernel_width is window, string is len of slide
receptive_field = calculate_receptive_field(conv_feature_layers)
print(f"{receptive_field=} samples")
print(f"{round(receptive_field / (sample_rate / 1000),3)} ms")

receptive_field=1328 samples
30.113 ms


In [45]:
conv = ConvFeatureExtractionModel(conv_feature_layers, mode="layer_norm")
print('num params: ', conv.params())
print('input shape=',x.shape)
conv_out = conv(x, verbose=True)
print("output shape= ", conv_out.shape)

num params:  23026176
input shape= torch.Size([10, 220500])
torch.Size([10, 1, 220500])
torch.Size([10, 768, 110246])
torch.Size([10, 768, 55119])
torch.Size([10, 768, 27555])
torch.Size([10, 768, 13777])
torch.Size([10, 768, 6888])
torch.Size([10, 768, 3443])
torch.Size([10, 768, 1721])
torch.Size([10, 768, 860])
torch.Size([10, 768, 430])
torch.Size([10, 768, 215])
output shape=  torch.Size([10, 768, 215])
